The following scripts intends to manually deploy required resources across different clusters.

# Remote Secret

In [14]:

import json
import subprocess
import base64
import sys
import os
from typing import List, Tuple, Dict, Optional


NAMES_FILE = "names"


def read_pairs(filename: str) -> List[Tuple[str, str]]:
    """Read whitespace-separated pairs from filename. Skip empty/comment lines."""
    pairs: List[Tuple[str, str]] = []
    with open(filename, "r", encoding="utf-8") as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith("#"):
                continue
            parts = line.split()
            if len(parts) < 2:
                print(f"Skipping malformed line (need at least 2 columns): {line}", file=sys.stderr)
                continue
            # cluster name is first column, current secret context is second
            pairs.append((parts[0], parts[1]))
    return pairs


def get_remote_secret_yaml(cluster_name: str) -> Optional[str]:
    """
    Fetch the secret from the local cluster and return decoded YAML string found in
    item[0].data["remote-secret.yaml"].
    Returns None if not found or on error.
    """
    label = f"skycluster.io/secret-type=cluster-cacert,skycluster.io/cluster-name={cluster_name}"
    cmd = ["kubectl", "get", "secrets", "-n", "skycluster-system", "-l", label, "-o", "json"]

    try:
        proc = subprocess.run(cmd, check=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to run kubectl get for cluster '{cluster_name}': {e}", file=sys.stderr)
        print(e.stderr.decode(errors="ignore"), file=sys.stderr)
        return None

    try:
        data = json.loads(proc.stdout)
    except json.JSONDecodeError as e:
        print(f"Failed to parse kubectl JSON for cluster '{cluster_name}': {e}", file=sys.stderr)
        return None

    items = data.get("items", [])
    if not items:
        print(f"No secrets found for cluster '{cluster_name}' (label search).", file=sys.stderr)
        return None

    first = items[0]
    secret_data = first.get("data", {})
    # The key used in the original jsonpath was remote-secret.yaml (with a dot).
    key = "remote-secret.yaml"
    if key not in secret_data:
        # if not present, try to find any key that ends with 'remote-secret.yaml' or endswith '.yaml'
        for k in secret_data.keys():
            if k.endswith("remote-secret.yaml") or k.endswith(".yaml"):
                key = k
                break
        else:
            print(f"Secret for cluster '{cluster_name}' does not contain 'remote-secret.yaml' in data keys: {list(secret_data.keys())}", file=sys.stderr)
            return None

    b64val = secret_data.get(key)
    if not b64val:
        print(f"No value for key '{key}' in secret for cluster '{cluster_name}'.", file=sys.stderr)
        return None

    try:
        decoded = base64.b64decode(b64val).decode("utf-8")
    except Exception as e:
        print(f"Failed to base64-decode remote secret for cluster '{cluster_name}': {e}", file=sys.stderr)
        return None

    return decoded


def apply_secret_to_context(secret_yaml: str, context: str, KUBECONFIG_PATH: str) -> bool:
    """Apply the given YAML to the provided kubectl context. Returns True on success."""
    cmd = ["kubectl", "--context", context, "apply", "-f", "-"]
    try:
        env = os.environ.copy()
        env["KUBECONFIG"] = KUBECONFIG_PATH
        proc = subprocess.run(cmd, input=secret_yaml.encode("utf-8"), capture_output=True, check=True, env=env)
        stdout = proc.stdout.decode("utf-8", errors="ignore").strip()
        stderr = proc.stderr.decode("utf-8", errors="ignore").strip()
        if stdout:
            print(f"[{context}] apply stdout:\n{stdout}")
        if stderr:
            print(f"[{context}] apply stderr:\n{stderr}", file=sys.stderr)
        return True
    except subprocess.CalledProcessError as e:
        print(f"[{context}] Failed to apply secret: returncode={e.returncode}", file=sys.stderr)
        if e.stdout:
            print(e.stdout.decode(errors="ignore"), file=sys.stderr)
        if e.stderr:
            print(e.stderr.decode(errors="ignore"), file=sys.stderr)
        return False



In [ ]:

# k get xkubes.bm | tail -n+2 | awk '{print $1}' > names
# k get xkubes.os | tail -n+2 | awk '{print $1}' >> names
# k get xkubes.aws | tail -n+2 | awk '{print $1}' >> names
# k get xkubes.gcp | tail -n+2 | awk '{print $1}' >> names
# echo "" > ctxs
# k config get-contexts | tail -n+2  >> ctxs

# rm ctxs_to_use
# while read r; do cat ./ctxs | grep "${r::-6}" | awk '{print $1}' >> ctxs_to_use;done < names

# # open ctx_to_use and match wuth names

In [ ]:
KUBECONFIG_PATH="/home/ubuntu/.kube/config:/tmp/kubeconfigs/kube-savi-toronto:/tmp/kubeconfigs/xk-aws-us-west--hxvbe:/tmp/kubeconfigs/xk-aws-eu-west--q28fg:/tmp/kubeconfigs/xk-aws-ap-east--12z23:/tmp/kubeconfigs/xk-aws-eu-north-6kfjn:/tmp/kubeconfigs/xk-aws-ap-north-8db1e:/tmp/kubeconfigs/xk-aws-us-east--us9a8:/tmp/kubeconfigs/xk-aws-ca-centr-gsa09:/tmp/kubeconfigs/xk-gcp-us-south-v2dvh:/tmp/kubeconfigs/xk-gcp-northame-oo44n:/tmp/kubeconfigs/xk-aws-eu-south-66hap:/tmp/kubeconfigs/kube-os-scinet:/tmp/kubeconfigs/xk-gcp-us-centr-e3d7y:/tmp/kubeconfigs/xk-gcp-us-east1-ny5jh:/tmp/kubeconfigs/xk-aws-ca-west--vzd0n:/tmp/kubeconfigs/xk-aws-us-east--guz6u:"

In [ ]:

NAMES_FILE="remote-names"
pairs = read_pairs(NAMES_FILE)
if not pairs:
    print(f"No valid pairs read from {NAMES_FILE}. Exiting.", file=sys.stderr)
    sys.exit(1)

# Build list of all contexts (the second column values)
all_contexts = [ctx for (_cluster, ctx) in pairs]

# For each pair, get the secret and apply to all other contexts (except itself)
for cluster_name, source_ctx in pairs:
    print(f"Processing cluster '{cluster_name}' (source context: '{source_ctx}')")
    secret_yaml = get_remote_secret_yaml(cluster_name)
    if not secret_yaml:
        print(f"Skipping cluster '{cluster_name}' due to missing secret.", file=sys.stderr)
        continue

    # apply to every context except source_ctx
    targets = [c for c in all_contexts if c != source_ctx]
    if not targets:
        print(f"No target contexts to apply for cluster '{cluster_name}'.")
        continue

    for target_ctx in targets:
        print(f"Applying secret from cluster '{cluster_name}' to context '{target_ctx}'...")
        ok = apply_secret_to_context(secret_yaml, target_ctx, KUBECONFIG_PATH)
        if ok:
            print(f"Applied to {target_ctx}")
        else:
            print(f"Failed to apply to {target_ctx}", file=sys.stderr)
    # break




# Application Manifests

In [7]:

import os
import shutil
import subprocess
import sys
import tempfile
from typing import Dict, Iterable, Optional, Tuple


try:
    import yaml
except Exception:
    print("ERROR: PyYAML is required. Install with: pip install pyyaml")
    sys.exit(2)



def ask_for_path(prompt, default):
    if os.path.exists(default):
        return default
    path = input(f"{prompt} (default: {default}): ").strip()
    if not path:
        path = default
    return path


def read_wrapper_yaml(path):
    """Read the YAML wrapper file and return a dict with 'manifest' and 'provider' keys."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Input YAML file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)
    if not isinstance(data, dict):
        raise ValueError("Input YAML does not contain a mapping at top level.")
    if "manifest" not in data or "provider" not in data:
        raise KeyError("Input YAML must contain top-level keys 'manifest' and 'provider'.")
    return data


def load_mapping_file(path):
    """Load mapping file where each non-empty non-comment line is:
       provider_key <whitespace> context_name
       Returns dict mapping provider_key -> context_name
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Mapping file not found: {path}")
    mapping = {}
    with open(path, "r", encoding="utf-8") as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith("#"):
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            key = parts[0]
            context = parts[1]
            mapping[key] = context
    return mapping


def strip_provider_prefix(provider_raw, prefix="k8s-eks-"):
    """Remove prefix (only once) if present."""
    if provider_raw.startswith(prefix):
        return provider_raw[len(prefix) :]
    if provider_raw.startswith("k8s-gke-"):
        return provider_raw[len("k8s-gke-") :]
    return provider_raw


def find_context_for_provider(provider_key, mapping):
    """Return context name for the provider_key or None if not found."""
    # if key contains the provider_key as substring of any mapping key, return that context
    for key, context in mapping.items():
        if provider_key in key or key in provider_key:
            return context


def write_manifest_to_tempfile(manifest_obj):
    """Dump the manifest object to a temporary YAML file and return its path."""
    fd, path = tempfile.mkstemp(prefix="manifest_", suffix=".yaml")
    os.close(fd)
    with open(path, "w", encoding="utf-8") as f:
        yaml.safe_dump(manifest_obj, f, default_flow_style=False)
    return path


def run_kubectl_apply(manifest_path, context):
    """Run kubectl apply -f manifest_path --context context. Returns (returncode, stdout, stderr)."""
    kubectl_path = shutil.which("kubectl")
    if not kubectl_path:
        raise EnvironmentError("kubectl not found in PATH. Please install or add it to PATH.")
    env = os.environ.copy()
    env["KUBECONFIG"] = KUBECONFIG_PATH
    cmd = [kubectl_path, "apply", "-f", manifest_path, "--context", context]
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, env=env)
    return proc.returncode, proc.stdout, proc.stderr



def run_shell_command(command: str, kubeconfig_path: Optional[str] = None) -> Tuple[int, str, str]:
    """
    Run a given shell command string and return (returncode, stdout, stderr).
    If kubeconfig_path is provided, set KUBECONFIG in the environment for the subprocess.
    """
    env = os.environ.copy()
    env["KUBECONFIG"] = KUBECONFIG_PATH

    proc = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, env=env)
    return proc.returncode, proc.stdout, proc.stderr


In [8]:
KUBECONFIG_PATH="/home/ubuntu/.kube/config:/tmp/kubeconfigs/kube-savi-toronto:/tmp/kubeconfigs/xk-aws-us-west--hxvbe:/tmp/kubeconfigs/xk-aws-eu-west--q28fg:/tmp/kubeconfigs/xk-aws-ap-east--12z23:/tmp/kubeconfigs/xk-aws-eu-north-6kfjn:/tmp/kubeconfigs/xk-aws-ap-north-8db1e:/tmp/kubeconfigs/xk-aws-us-east--us9a8:/tmp/kubeconfigs/xk-aws-ca-centr-gsa09:/tmp/kubeconfigs/xk-gcp-us-south-v2dvh:/tmp/kubeconfigs/xk-gcp-northame-oo44n:/tmp/kubeconfigs/xk-aws-eu-south-66hap:/tmp/kubeconfigs/kube-os-scinet:/tmp/kubeconfigs/xk-gcp-us-centr-e3d7y:/tmp/kubeconfigs/xk-gcp-us-east1-ny5jh:/tmp/kubeconfigs/xk-aws-ca-west--vzd0n:/tmp/kubeconfigs/xk-aws-us-east--guz6u:"

In [ ]:
ns_files = """
ns-air-quality-aws-a-1vt76.yaml
ns-air-quality-aws-a-ycn90.yaml
ns-air-quality-aws-c-2qm6v.yaml
ns-air-quality-aws-c-n0td4.yaml
ns-air-quality-aws-e-9j20v.yaml
ns-air-quality-aws-e-dpc4q.yaml
ns-air-quality-aws-e-k5n1b.yaml
ns-air-quality-aws-u-h6o7e.yaml
ns-air-quality-aws-u-manuj.yaml
ns-air-quality-aws-u-nh5wp.yaml
ns-air-quality-gcp-n-fmk7f.yaml
ns-air-quality-gcp-u-a5sym.yaml
ns-air-quality-gcp-u-aebmn.yaml
ns-air-quality-gcp-u-enttq.yaml
ns-air-quality-savi--2njba.yaml
ns-air-quality-savi--xhz29.yaml
ns-air-quality-savi--zco7a.yaml
"""


In [72]:
dp_files = """
dp-central-storage-a-gryxc.yaml
dp-collector-aws-ap--p2ynz.yaml
dp-collector-aws-ap--wqmka.yaml
dp-collector-aws-ca--2mb6p.yaml
dp-collector-aws-ca--ctn77.yaml
dp-collector-aws-eu--5y9cf.yaml
dp-collector-aws-eu--fq8lu.yaml
dp-collector-aws-eu--vnspi.yaml
dp-collector-aws-us--7tdxm.yaml
dp-collector-aws-us--tdliz.yaml
dp-collector-edge-sa-38uuj.yaml
dp-collector-edge-sa-9wny2.yaml
dp-collector-edge-sa-b8y89.yaml
dp-collector-gcp-nor-1cvu9.yaml
dp-collector-gcp-us--7bk5c.yaml
dp-collector-gcp-us--7yfyu.yaml
dp-collector-gcp-us--dsgj8.yaml
dp-dashboard-aws-us--l6xhn.yaml
dp-ingestor-aws-ap-e-620al.yaml
dp-ingestor-aws-ap-n-6nn4x.yaml
dp-ingestor-aws-ca-c-vaqtz.yaml
dp-ingestor-aws-ca-w-tpeln.yaml
dp-ingestor-aws-eu-s-bf280.yaml
dp-ingestor-aws-eu-w-vrr4k.yaml
dp-ingestor-aws-us-e-agldo.yaml
dp-ingestor-aws-us-e-zxtla.yaml
dp-ingestor-aws-us-w-a1gb7.yaml
dp-ingestor-gcp-nort-2ee9p.yaml
dp-ingestor-gcp-us-e-pulmr.yaml
dp-local-storage-aws-7hp1a.yaml
dp-local-storage-aws-7xr5k.yaml
dp-local-storage-aws-cqu8z.yaml
dp-local-storage-aws-jot81.yaml
dp-local-storage-aws-zf5en.yaml
dp-processor-aws-ap--960qj.yaml
dp-processor-aws-ca--pf5q4.yaml
dp-processor-aws-eu--e5dxo.yaml
dp-processor-aws-us--54gen.yaml
dp-processor-aws-us--fhp6d.yaml
"""

In [25]:
svc_files = """
svc-svc-central-stor-0dsqw.yaml
svc-svc-central-stor-41j5u.yaml
svc-svc-central-stor-6uqfv.yaml
svc-svc-central-stor-74tu5.yaml
svc-svc-central-stor-7kaz9.yaml
svc-svc-central-stor-7zxmj.yaml
svc-svc-central-stor-8mukb.yaml
svc-svc-central-stor-b18o1.yaml
svc-svc-central-stor-e8coi.yaml
svc-svc-central-stor-htcqg.yaml
svc-svc-central-stor-j9fqr.yaml
svc-svc-central-stor-k6tds.yaml
svc-svc-central-stor-lv57r.yaml
svc-svc-central-stor-n1dv9.yaml
svc-svc-central-stor-v3wbv.yaml
svc-svc-central-stor-vujao.yaml
svc-svc-central-stor-zg4o9.yaml
svc-svc-collector-aw-0fk2e.yaml
svc-svc-collector-aw-3fzyy.yaml
svc-svc-collector-aw-dcm11.yaml
svc-svc-collector-aw-jhchg.yaml
svc-svc-collector-aw-k71vm.yaml
svc-svc-collector-aw-m0p4a.yaml
svc-svc-collector-aw-oogxe.yaml
svc-svc-collector-aw-u2uab.yaml
svc-svc-collector-aw-wqtjh.yaml
svc-svc-collector-aw-yjb4b.yaml
svc-svc-collector-gc-7m3rl.yaml
svc-svc-collector-gc-jfd99.yaml
svc-svc-collector-gc-tfvaj.yaml
svc-svc-collector-gc-z9fza.yaml
svc-svc-collector-sa-7acvt.yaml
svc-svc-collector-sa-hqeie.yaml
svc-svc-collector-sa-ug79r.yaml
svc-svc-dashboard-aw-4ngk9.yaml
svc-svc-dashboard-aw-albid.yaml
svc-svc-dashboard-aw-g46k2.yaml
svc-svc-dashboard-aw-ivnfl.yaml
svc-svc-dashboard-aw-lebcl.yaml
svc-svc-dashboard-aw-qg9nf.yaml
svc-svc-dashboard-aw-t6s1o.yaml
svc-svc-dashboard-aw-tis3x.yaml
svc-svc-dashboard-aw-wck6f.yaml
svc-svc-dashboard-aw-zh5cx.yaml
svc-svc-dashboard-gc-mtllc.yaml
svc-svc-dashboard-gc-ntufx.yaml
svc-svc-dashboard-gc-sg07v.yaml
svc-svc-dashboard-gc-syelr.yaml
svc-svc-dashboard-sa-2azkg.yaml
svc-svc-dashboard-sa-crf20.yaml
svc-svc-dashboard-sa-d2pjb.yaml
svc-svc-ingestor-aws-16clc.yaml
svc-svc-ingestor-aws-60wgn.yaml
svc-svc-ingestor-aws-8sjh5.yaml
svc-svc-ingestor-aws-buv19.yaml
svc-svc-ingestor-aws-jym63.yaml
svc-svc-ingestor-aws-p0t0l.yaml
svc-svc-ingestor-aws-rzryr.yaml
svc-svc-ingestor-aws-vua9z.yaml
svc-svc-ingestor-aws-x4zde.yaml
svc-svc-ingestor-aws-yiv72.yaml
svc-svc-ingestor-gcp-2swcw.yaml
svc-svc-ingestor-gcp-6xqfz.yaml
svc-svc-ingestor-gcp-mwnet.yaml
svc-svc-ingestor-gcp-sipkh.yaml
svc-svc-ingestor-sav-iguu1.yaml
svc-svc-ingestor-sav-xt4p3.yaml
svc-svc-ingestor-sav-zo4g1.yaml
svc-svc-local-storag-2wx4q.yaml
svc-svc-local-storag-3duk2.yaml
svc-svc-local-storag-4f20r.yaml
svc-svc-local-storag-4zlpt.yaml
svc-svc-local-storag-5cbh7.yaml
svc-svc-local-storag-6i1qe.yaml
svc-svc-local-storag-8s34h.yaml
svc-svc-local-storag-9vauu.yaml
svc-svc-local-storag-ag0uh.yaml
svc-svc-local-storag-lq8ft.yaml
svc-svc-local-storag-m6zn1.yaml
svc-svc-local-storag-pffww.yaml
svc-svc-local-storag-qcedr.yaml
svc-svc-local-storag-rmy4p.yaml
svc-svc-local-storag-tv4ni.yaml
svc-svc-local-storag-xv6ot.yaml
svc-svc-local-storag-z1j4w.yaml
svc-svc-processor-aw-1gtfu.yaml
svc-svc-processor-aw-3gg4q.yaml
svc-svc-processor-aw-6a7p6.yaml
svc-svc-processor-aw-6mj0c.yaml
svc-svc-processor-aw-a18iu.yaml
svc-svc-processor-aw-begx7.yaml
svc-svc-processor-aw-bjvhu.yaml
svc-svc-processor-aw-lnrqv.yaml
svc-svc-processor-aw-yl952.yaml
svc-svc-processor-aw-zog21.yaml
svc-svc-processor-gc-f911q.yaml
svc-svc-processor-gc-spwcw.yaml
svc-svc-processor-gc-vzdm9.yaml
svc-svc-processor-gc-wu5uj.yaml
svc-svc-processor-sa-0ae54.yaml
svc-svc-processor-sa-4xmi3.yaml
svc-svc-processor-sa-q8f4c.yaml
"""

In [ ]:
vs_svc_files="""
vs-svc-central-storage-aw-3in6k.yaml
vs-svc-central-storage-aw-ax5oo.yaml
vs-svc-central-storage-aw-cy32m.yaml
vs-svc-central-storage-aw-faal4.yaml
vs-svc-central-storage-aw-ujbm8.yaml
vs-svc-ingestor-aws-ap-ea-p6ir7.yaml
vs-svc-ingestor-aws-ap-no-a57sw.yaml
vs-svc-ingestor-aws-ca-ce-lfv91.yaml
vs-svc-ingestor-aws-ca-we-z8lo7.yaml
vs-svc-ingestor-aws-eu-no-ttc37.yaml
vs-svc-ingestor-aws-eu-so-0bfbn.yaml
vs-svc-ingestor-aws-eu-we-s8h9l.yaml
vs-svc-ingestor-aws-us-ea-ysjxi.yaml
vs-svc-ingestor-aws-us-we-watsq.yaml
vs-svc-ingestor-gcp-north-5lqv2.yaml
vs-svc-ingestor-gcp-us-ce-49hm2.yaml
vs-svc-ingestor-gcp-us-ea-fhh6x.yaml
vs-svc-ingestor-gcp-us-so-v2tda.yaml
vs-svc-ingestor-savi-scin-ftc5x.yaml
vs-svc-ingestor-savi-toro-zsd24.yaml
vs-svc-ingestor-savi-vaug-8h96n.yaml
vs-svc-local-storage-aws--959gp.yaml
vs-svc-local-storage-aws--9g9fw.yaml
vs-svc-local-storage-aws--c382g.yaml
vs-svc-local-storage-aws--cv7vx.yaml
vs-svc-local-storage-aws--ejpt8.yaml
vs-svc-local-storage-aws--ghi02.yaml
vs-svc-local-storage-aws--iw2df.yaml
vs-svc-local-storage-aws--p4mgr.yaml
vs-svc-local-storage-aws--r3eey.yaml
vs-svc-local-storage-gcp--7nik8.yaml
vs-svc-local-storage-gcp--ea3zt.yaml
vs-svc-processor-aws-ap-n-pvo0f.yaml
vs-svc-processor-aws-ca-c-n9ebh.yaml
vs-svc-processor-aws-eu-w-94rkr.yaml
vs-svc-processor-aws-us-e-6hahw.yaml
vs-svc-processor-aws-us-w-k366u.yaml
"""

In [77]:
dr_files = """
dr-svc-central-storage-aw-1kyon.yaml
dr-svc-central-storage-aw-ha9h8.yaml
dr-svc-central-storage-aw-pczdt.yaml
dr-svc-central-storage-aw-sbb2p.yaml
dr-svc-central-storage-aw-t7sdr.yaml
dr-svc-ingestor-aws-ap-ea-mgxlo.yaml
dr-svc-ingestor-aws-ap-no-9iier.yaml
dr-svc-ingestor-aws-ca-ce-fkoqe.yaml
dr-svc-ingestor-aws-ca-we-m64z1.yaml
dr-svc-ingestor-aws-eu-no-zjkez.yaml
dr-svc-ingestor-aws-eu-so-6yk85.yaml
dr-svc-ingestor-aws-eu-we-azlrz.yaml
dr-svc-ingestor-aws-us-ea-dafxd.yaml
dr-svc-ingestor-aws-us-we-49fks.yaml
dr-svc-ingestor-gcp-north-3cifk.yaml
dr-svc-ingestor-gcp-us-ce-y195j.yaml
dr-svc-ingestor-gcp-us-ea-t7mlm.yaml
dr-svc-ingestor-gcp-us-so-isf5s.yaml
dr-svc-ingestor-savi-scin-n3zt9.yaml
dr-svc-ingestor-savi-toro-bxvca.yaml
dr-svc-ingestor-savi-vaug-aafqa.yaml
dr-svc-local-storage-aws--1o8au.yaml
dr-svc-local-storage-aws--2z0yl.yaml
dr-svc-local-storage-aws--jp4jc.yaml
dr-svc-local-storage-aws--odby0.yaml
dr-svc-local-storage-aws--ofols.yaml
dr-svc-local-storage-aws--rv9op.yaml
dr-svc-local-storage-aws--sfapx.yaml
dr-svc-local-storage-aws--uoul7.yaml
dr-svc-local-storage-aws--wv25c.yaml
dr-svc-local-storage-gcp--gp6ym.yaml
dr-svc-local-storage-gcp--m13wk.yaml
dr-svc-processor-aws-ap-n-ihfer.yaml
dr-svc-processor-aws-ca-c-2jhzj.yaml
dr-svc-processor-aws-eu-w-ipy3z.yaml
dr-svc-processor-aws-us-e-hg41m.yaml
dr-svc-processor-aws-us-w-jn9ez.yaml
"""

In [78]:
input_files = dr_files

In [12]:
input_files = """
dp-ingestor-aws-ap-e-620al.yaml
dp-ingestor-aws-ap-n-6nn4x.yaml
dp-ingestor-aws-ca-c-vaqtz.yaml
dp-ingestor-aws-ca-w-tpeln.yaml
dp-ingestor-aws-eu-s-bf280.yaml
dp-ingestor-aws-eu-w-vrr4k.yaml
dp-ingestor-aws-us-e-agldo.yaml
dp-ingestor-aws-us-e-zxtla.yaml
dp-ingestor-aws-us-w-a1gb7.yaml
dp-ingestor-gcp-nort-2ee9p.yaml
dp-ingestor-gcp-us-e-pulmr.yaml
"""

In [ ]:

folder = "/tmp/istio2/"
mapping_path = "remote-names"

filenames = [ln.strip() for ln in input_files.splitlines() if ln.strip()]
for fname in filenames:
    full_path = os.path.join(folder, fname)
    data = read_wrapper_yaml(full_path)
    manifest_obj = data["manifest"]
    provider_raw = str(data["provider"]).strip()
    print(provider_raw)
    if provider_raw == "":
        continue

    provider_key = strip_provider_prefix(provider_raw, prefix="k8s-eks-")
    provider_key = provider_key[:-6]
    print(f"Using provider key (prefix stripped): {provider_key}")

    mapping = load_mapping_file(mapping_path)

    context = find_context_for_provider(provider_key, mapping)
    print(f"Resolved kubectl context: {context}")
    if context is None:
        print(f" ==> Could not find context for provider key '{provider_key}'. Skipping.", file=sys.stderr)
        continue

    tmp_manifest = write_manifest_to_tempfile(manifest_obj)

    code, out, err = run_kubectl_apply(tmp_manifest, context)
    print(out.strip())
    print(err.strip())
    print("----------")
    
    if code != 0:
        print(f"kubectl exited with code {code}")
        sys.exit(code)

# Kiali Remote Secret

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# kube-os-scinet-hxn2s        br2
CTX_NAME = """
kube-savi-toronto-hc4cq        op1
xk-aws-ap-east--12z23        aws3
xk-aws-ap-north-8db1e        aws4
xk-aws-ca-centr-gsa09        aws5
xk-aws-ca-west--vzd0n        aws6
xk-aws-eu-north-6kfjn        aws7
xk-aws-eu-south-66hap        aws8
xk-aws-eu-west--q28fg        aws9
xk-aws-us-east--guz6u        aws10
xk-aws-us-east--us9a8        aws11
xk-aws-us-west--hxvbe        aws12
xk-gcp-northame-oo44n-77zlf        gcp1
xk-gcp-us-centr-e3d7y-jsctz        gcp2
xk-gcp-us-east1-ny5jh-9q5ff        gcp3
xk-gcp-us-south-v2dvh-cvvqm        gcp4
"""

# parse into (context, cluster_name) pairs
lines = [ln.strip() for ln in CTX_NAME.splitlines() if ln.strip()]
ctx_pairs = [tuple(ln.split(None, 1)) for ln in lines]  # (context, cluster_name)

kiali_remote = "/home/ubuntu/kiali-prepare-remote-cluster.sh"

def install(name, cluster_name):
    cmd = (
        f"{kiali_remote} --remote-cluster-name {cluster_name} "
        f"--process-remote-resources true "
        f"--process-kiali-secret true "
        f"--kiali-cluster-namespace istio-system "
        f"--kiali-cluster-context kind-skycluster  --remote-cluster-context {name} "
    )
    try:
        code, out, err = run_shell_command(cmd, kubeconfig_path=KUBECONFIG_PATH)
    except Exception as e:
        return {"name": name, "cluster": cluster_name, "code": -1, "out": "", "err": str(e)}
    return {"name": name, "cluster": cluster_name, "code": code, "out": out, "err": err}

max_workers = min(10, len(ctx_pairs))
with ThreadPoolExecutor(max_workers=max_workers) as exe:
    futures = {exe.submit(install, ctx_name, cluster_name): (ctx_name, cluster_name)
               for cluster_name, ctx_name in ctx_pairs}
    for fut in as_completed(futures):
        res = fut.result()
        print(f"Installing remote-secret kiali in context: {res['name']} (cluster: {res['cluster']})")
        print("return code:", res["code"])
        print(res["out"])
        print(res["err"])
        print("------")

# Helm Prometheus

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

CONTEXTS="""
aws3
aws4
aws5
aws6
aws7
aws8
aws9
aws10
aws11
aws12
gcp1
gcp2
gcp3
gcp4
op1
"""

ctx = [ln.strip() for ln in CONTEXTS.splitlines() if ln.strip()]
prom_values_file = "/home/ubuntu/skycluster-project/skycluster-operator/examples/dev/prometheus/prometheous-thanos-values.yaml"


def install(name):
    cmd = (
        f"helm install {name} prometheus-community/kube-prometheus-stack "
        f"-f {prom_values_file} -n monitoring "
        f"--create-namespace --kube-context {name}"
    )
    try:
        code, out, err = run_shell_command(cmd, kubeconfig_path=KUBECONFIG_PATH)
    except Exception as e:
        return {"name": name, "code": -1, "out": "", "err": str(e)}
    return {"name": name, "code": code, "out": out, "err": err}


max_workers = min(10, len(ctx))  # tune as needed
with ThreadPoolExecutor(max_workers=max_workers) as exe:
    futures = {exe.submit(install, name): name for name in ctx}
    for fut in as_completed(futures):
        res = fut.result()
        print(f"Installing prometheus-thanos in context: {res['name']}")
        print("return code:", res["code"])
        print(res["out"])
        print(res["err"])
        print("------")


In [ ]:

CONTEXTS="""
aws3
aws4
aws5
aws6
aws7
aws8
aws9
aws10
aws11
aws12
gcp1
gcp2
gcp3
gcp4
op1
"""

ctx = [ln.strip() for ln in CONTEXTS.splitlines() if ln.strip()]

def run_this(name):
    cmd = (
        f"kubectl --context {name} get svc {name}-kube-prometheus-stack-thanos-external -n monitoring "
        f"-o jsonpath='{{.spec.clusterIP}}'"
    )
    try:
        code, out, err = run_shell_command(cmd, kubeconfig_path=KUBECONFIG_PATH)
    except Exception as e:
        return {"name": name, "code": -1, "out": "", "err": str(e)}
    return {"name": name, "code": code, "out": out, "err": err}

for name in ctx:
    res = run_this(name)
    print(f"{name}: {res['out']}")


# Service Monitor

In [ ]:

CONTEXTS="""
aws3
aws4
aws5
aws6
aws7
aws8
aws9
aws10
aws11
aws12
gcp1
gcp2
gcp3
gcp4
op1
"""

# FILE="/home/ubuntu/skycluster-project/skycluster-operator/examples/apps/air-quality/service-monitor.yaml"
FILE="/home/ubuntu/skycluster-project/skycluster-operator/examples/dev/prometheus/prometheus-scrape-config-remote.yaml"
ctx = [ln.strip() for ln in CONTEXTS.splitlines() if ln.strip()]

def run_this(name):
    cmd = (
        f"kubectl --context {name} apply -f {FILE} "
    )
    try:
        code, out, err = run_shell_command(cmd, kubeconfig_path=KUBECONFIG_PATH)
    except Exception as e:
        return {"name": name, "code": -1, "out": "", "err": str(e)}
    return {"name": name, "code": code, "out": out, "err": err}

for name in ctx:
    res = run_this(name)
    print(f"{name}: {res}")
